In [2]:
# https://www.geeksforgeeks.org/deploy-machine-learning-model-using-flask/#:~:text=%23%20prediction%20function,prediction%20%3D%20prediction)
# importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import datetime
from flask import Flask, jsonify, request
from flask import Flask, render_template


import flask
app = Flask(__name__)


def dwy(x):
    x['day'] = pd.DatetimeIndex(x['Date']).day
    x['Week'] = pd.DatetimeIndex(x['Date']).week
    x['year'] = pd.DatetimeIndex(x['Date']).year
    
    return x

def isholiday_le(x):
    IsHoliday = []
    for i in x['IsHoliday']:
        if i == False:
            IsHoliday.append(0)
        else:
            IsHoliday.append(1)
    x['IsHoliday'] = IsHoliday
    return x

def type_le(x):
    store_type = []
    for i in x['Type']:
        if i == 'A':
            store_type.append(1)
        elif i == 'B':
            store_type.append(2)
        else:
            store_type.append(3)
    x['Type'] = store_type
    return x

final_model = joblib.load('final_model.pkl')


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():    
    store_data = pd.read_csv('stores.csv')
    
    df = pd.DataFrame()
    df['Store'] = list([int(request.form['Store'])])
    df['Dept'] = list([int(request.form['Dept'])])
    df['Date'] = list([request.form['Date']])
    df['IsHoliday'] = list([request.form['IsHoliday']]) 
    
    
    data = df.merge(store_data, on=['Store'], how='inner')
    
    
    data = dwy(data)
    data = isholiday_le(data)
    data = type_le(data)
    
    data = data[['Dept', 'Size', 'Store', 'Type', 'Week', 'day', 'IsHoliday', 'year']]
    
    
    predicted_weekly_sale = final_model.predict(data)
    
    final_result = np.round(predicted_weekly_sale, 2)

    return render_template("index.html", prediction_text = 'Weekly Sales in week {} will be ${}'.format(list(data['Week'])[0], final_result[0]))


if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
